# 爱科研-人工智能科研实训项目
## 卷积神经网络（Convolutional Neural Network, CNN）

## 项目：计算机视觉 - 交通标志识别

在这个notebook文件中，有些模板代码已经提供给你，但你还需要实现更多的功能来完成这个项目。除非有明确要求，你无须修改任何已给出的代码。以**'(练习)'**开始的标题表示接下来的代码部分中有你需要实现的功能。这些部分都配有详细的指导，需要实现的部分也会在注释中以'TODO'标出。请仔细阅读所有的提示。

除了实现代码外，你还**需要**回答一些与项目及代码相关的问题。每个需要回答的问题都会以 **'问题 X'** 标记。请仔细阅读每个问题，并且在问题后的 **'回答'** 部分写出完整的答案。我们将根据 你对问题的回答 和 撰写代码实现的功能 来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 让我们开始吧
在这个notebook中，你将迈出第一步，来开发可以作为自动驾驶汽车视觉一部分的算法。在现实世界中，你需要拼凑一系列的模型来完成不同的任务；举个例子，你需要先对摄像机获取的图片进行交通标志的探测，再对其标志类别进行识别。在做项目的过程中，你可能会遇到不少失败的预测，因为并不存在完美的算法和模型。你最终提交的不完美的解决方案也一定会给你带来一个有趣的学习经验！

![Sample Dog Output](images/preview.png)

### 项目内容

我们将这个notebook分为不同的步骤，你可以使用下面的链接来浏览此notebook。

* [Step 0](#step0): 图像数据预览
* [Step 1](#step1): 导入数据集
* [Step 2](#step2): 数据增强
* [Step 3](#step3): CNN模型构建
* [Step 4](#step4): 模型训练
* [Step 5](#step5): 测试你的算法

在该项目中包含了如下的问题：

* [问题 1](#question1): 使用类别不均衡的训练样本进行机器学习模型的训练，可能会产生哪些效果？
* [问题 2](#question2): 简要概括早期停止与学习率衰减的原理。
* [问题 3](#question3): 思考：采用哪些方法可以进一步提高模型的准确度？

---
<a id='step0'></a>
## 步骤 0: 数据预览

### 图像数据

图像数据格式为 `ppm`，无法直接预览，我们使用 `PIL` 库中的 `Image` 函数对 `ppm` 格式图像进行处理。

In [ ]:
import numpy as np
import random as rd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from pandas import read_csv
from PIL import Image as pil_image
%matplotlib inline

def show_sign(imgs, tags, per_row=2):
    n    = len(imgs)
    rows = (n + per_row - 1)//per_row
    cols = min(per_row, n)
    fig, axes = plt.subplots(rows,cols, figsize=(24//per_row*cols,24//per_row*rows))
    for ax in axes.flatten(): ax.axis('off')
    for i,(img,ax) in enumerate(zip(imgs, axes.flatten())): 
        ax.imshow(img.convert('RGB'))
        ax.set_title(tags[i])

### 随机展示100张训练集图片
trainlist = glob('./data/train/*/*')
example = [rd.randint(0, len(trainlist)-1) for _ in range(100)]
imgs = [pil_image.open(trainlist[i]) for i in example]
tags = [str(int(trainlist[i][-15:-10])) for i in example]
show_sign(imgs, tags, per_row=10)

### 标签数据

下面我们看看每个数字ID分别对应的交通标志内容。该信息储存在 `./data/signnames.csv` 中， 我们使用 `pandas` 读取并储存该信息于字典 `sign_names` 中。

In [ ]:
sign_names = dict([(p,w) for _,p,w in read_csv('./data/signnames.csv').to_records()])

sign_name_table = PrettyTable()
sign_name_table.field_names = ['class value', 'Name of Traffic sign']
for i, j in sign_names.items(): sign_name_table.add_row([i, j])

print(sign_name_table)

---
<a id='step1'></a>
## 步骤1：导入数据集

在下方的代码单元（cell）中，我们以灰度图的形式导入了交通标志图像的数据集：
- 我们将图像尺寸统一调整为 32$\times$32
- `train['features']`, `test['features']` - 包含图像文件的numpy数组
- `train['label']` - 包含分类标签的数组

In [ ]:
SIZE = (32,32)
n_class = len(sign_names)

### 导入训练集
train = {}
train['features'] = []
train['label'] = []

print('Loading the training set:')
for sign in tqdm(range(n_class)):
    path = './data/train/' + str(sign).zfill(5) + '/*'
    for i in glob(path):
        img = pil_image.open(i)
        img = img.convert('L')  # 将图像转变为灰度图
        train['features'].append(np.array(img.resize(SIZE)))
        train['label'].append(sign)
train['label'] = np.array(train['label'])

### 导入测试集
test = {}
test['features'] = []
test['label'] = []

print('Loading the testing set:')
path = './data/test/*'
for i in tqdm(glob(path)):
    img = pil_image.open(i)
    img = img.convert('L')
    test['features'].append(np.array(img.resize(SIZE)))

n_train, n_test = len(train['features']), len(test['features'])

### 对数据特征进行维度匹配以便导入神经网络
train['features'] = np.expand_dims(train['features'], axis=-1)
test['features'] = np.expand_dims(test['features'], axis=-1)

print('Total number of classes:{}'.format(n_class))
print('Number of training examples =',n_train)
print('Number of testing examples =',n_test)
print('Image data shape=',train['features'].shape[1:])

---
<a id='step2'></a>
## 步骤2：数据增强

### 数据分布

通过频率直方图检查各个类别的数据量是否均衡

In [ ]:
def get_count_imgs_per_class(y, verbose=False):
    num_classes = len(np.unique(y))
    count_imgs_per_class = np.zeros(num_classes)

    for this_class in range(num_classes):
        if verbose: 
            print('class {} | count {}'.format(this_class, np.sum(y==this_class)))
        count_imgs_per_class[this_class] = np.sum(y==this_class)
    return count_imgs_per_class

class_freq = get_count_imgs_per_class(train['label'])
print('------- ')
print('Highest count: {} (class {})'.format(int(np.max(class_freq)), np.argmax(class_freq)))
print('Lowest count: {} (class {})'.format(int(np.min(class_freq)), np.argmin(class_freq)))
print('------- ')
plt.bar(np.arange(n_class), class_freq , align='center')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.xlim([-1, n_class])
plt.title("class frequency in Training set")
plt.show()

### 数据增强

这里的数据增强主要是：
- 增加训练集的大小 
- 调整了类别分布（由上图可以看出类别分布是不均衡的） 

>**Note：关于机器学习分类问题中的训练样本不均衡问题，可参考CSDN博客文章：[在分类中如何处理训练集中不平衡问题](https://blog.csdn.net/heyongluoyao8/article/details/49408131)

---
<a id='question1'></a>  

### __问题 1:__ 

使用类别不均衡的训练样本进行机器学习模型的训练，可能会产生哪些效果？

__回答:__ 

---

数据增强后，我们得到每个类别2000张图片 数据增强的方法主要就是从原始数据集中随机选取图片，并应用仿射变换。
仿射变换的限制条件为：
- 旋转角度我限制在 `[-10，10]` 度之间，如果旋转角度过大，有些交通标志的意思可能就会发生变化
- 水平、垂直移动的话，范围限制在 `[-3, 3]` 像素之间
- 伸缩变换限制在 `[0.8, 1.2]` 之间

In [ ]:
from skimage import transform as transf

### 仿射变换函数
def random_transform(img,angle_range=[-10,10],
                    scale_range=[0.8,1.2],
                    translation_range=[-3,3]):
    '''
    The function takes an image and performs a set of random affine transformation.
    img:original images
    ang_range:angular range of the rotation [-15,+15] deg for example
    scale_range: [0.8,1.2]
    shear_range:[10,-10]
    translation_range:[-2,2]
    '''
    img_height,img_width,img_depth = img.shape
    # Generate random parameter values
    angle_value = np.random.uniform(low=angle_range[0],high=angle_range[1],size=None)
    scaleX = np.random.uniform(low=scale_range[0],high=scale_range[1],size=None)
    scaleY = np.random.uniform(low=scale_range[0],high=scale_range[1],size=None)
    translationX = np.random.randint(low=translation_range[0],high=translation_range[1]+1,size=None)
    translationY = np.random.randint(low=translation_range[0],high=translation_range[1]+1,size=None)

    center_shift = np.array([img_height,img_width])/2. - 0.5
    transform_center = transf.SimilarityTransform(translation=-center_shift)
    transform_uncenter = transf.SimilarityTransform(translation=center_shift)

    transform_aug = transf.AffineTransform(rotation=np.deg2rad(angle_value),
                                          scale=(1/scaleY,1/scaleX),
                                          translation = (translationY,translationX))
    #Image transformation : includes rotation ,shear,translation,zoom
    full_tranform = transform_center + transform_aug + transform_uncenter
    new_img = transf.warp(img,full_tranform,preserve_range=True)

    return new_img.astype('uint8')

### 数据增强函数
def data_augmentation(X_dataset,y_dataset,augm_nbr,keep_dist=True):
    '''
    X_dataset:image dataset to augment
    y_dataset:label dataset
    keep_dist - True:keep class distribution of original dataset,
                False:balance dataset
    augm_param - is the augmentation parameter
                if keep_dist is True,increase the dataset by the factor 'augm_nbr' (2x,5x or 10x...)
                if keep_dist is False,make all classes have same number of images:'augm_nbr'(2500,3000 or 4000 imgs)
    '''
    X_train_dtype = X_dataset
    n_classes = len(np.unique(y_dataset))
    _,img_height,img_width,img_depth = X_dataset.shape
    class_freq = get_count_imgs_per_class(y_train)

    if keep_dist:
        extra_imgs_per_class = np.array([augm_nbr*x for x in get_count_imgs_per_class(y_dataset)])
    else:
        assert (augm_nbr>np.argmax(class_freq)),'augm_nbr must be larger than the height class count'
        extra_imgs_per_class = augm_nbr - get_count_imgs_per_class(y_dataset)

    total_extra_imgs = np.sum(extra_imgs_per_class)

    #if extra data is needed->run the dataaumentation op
    if total_extra_imgs > 0:
        X_extra = np.zeros((int(total_extra_imgs),img_height,img_width,img_depth),dtype=X_dataset.dtype)
        y_extra = np.zeros(int(total_extra_imgs))
        start_idx = 0
        #print('start data augmentation.....')
        for this_class in range(n_classes):
            #print('\t Class {}|Number of extra imgs{}'.format(this_class,int(extra_imgs_per_class[this_class])))
            n_extra_imgs = extra_imgs_per_class[this_class]
            end_idx = start_idx + n_extra_imgs

            if n_extra_imgs > 0:
                #get ids of all images belonging to this_class
                all_imgs_id = np.argwhere(y_dataset==this_class)
                new_imgs_x = np.zeros((int(n_extra_imgs),img_height,img_width,img_depth))

                for k in range(int(n_extra_imgs)):
                    #randomly pick an original image belonging to this class
                    rand_id = np.random.choice(all_imgs_id[0],size=None,replace=True)
                    rand_img = X_dataset[rand_id]
                    #Transform image
                    new_img = random_transform(rand_img)
                    new_imgs_x[k,:,:,:] = new_img
                #update tensors with new images and associated labels
                X_extra[int(start_idx):int(end_idx)] = new_imgs_x
                y_extra[int(start_idx):int(end_idx)] = np.ones((int(n_extra_imgs),))*this_class
                start_idx = end_idx
        return [X_extra,y_extra]
    else:
        return [None,None]

In [ ]:
# TODO: import train_test_split from sklearn


# TODO: split the data into training and validation subsets
X_train,X_val,y_train,y_val= (None, None, None, None)

print('*** Before data augmentation:')
print('Train set size:{}|Validation set size:{}\n'.format(X_train.shape[0],X_val.shape[0]))

X_extra,y_extra = data_augmentation(X_train,y_train,augm_nbr=2000,keep_dist=False)

if X_extra is not None:
    X_train = np.concatenate((X_train,X_extra.astype('uint8')),axis=0)
    y_train = np.concatenate((y_train,y_extra),axis=0)
    del X_extra,y_extra

print('*** After data augmentation:')
print('Train set size:{}|Validation set size:{}\n'.format(X_train.shape[0],X_val.shape[0]))

with mpl.rc_context(rc={'font.family': 'serif', 'font.size': 11}):
    fig = plt.figure(figsize=(9.5,3.5))
    ax1 = fig.add_subplot(121)
    plt.bar(np.arange(n_class),get_count_imgs_per_class(y_train),align='center')
    ax1.set_xlabel('Class')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Training Set')
    plt.xlim([-1,43])
    ax2 = fig.add_subplot(122)
    plt.bar(np.arange(n_class),get_count_imgs_per_class(y_val),align='center')
    ax2.set_xlabel('Class')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Validation Set')
    plt.xlim([-1,43])

---
<a id='step3'></a>
## 步骤 3: CNN模型构建

在对图像进行增强之后，我们需要更进一步的方法，来对标志的类别进行识别。在这一步中，你需要实现一个卷积神经网络来对交通标志图像进行分类。你需要从头实现你的卷积神经网络。

需要注意的是，在添加卷积层的时候，注意不要加上太多的（可训练的）层。更多的参数意味着更长的训练时间，也就是说你更可能需要一个 GPU 来加速训练过程。万幸的是，Keras 提供了能够轻松预测每次迭代（epoch）花费时间所需的函数。你可以据此推断你算法所需的训练时间。

### 数据预处理

由于图像像素值分布在 `0-255` 之间，数值较大，因此在将数据导入神经网络之前，我们需要对图像数组进行归一化处理。同时我们需要将标签进行独热编码。

In [ ]:
from keras.utils.np_utils import to_categorical

def preprocessed(dataset):
    n_imgs,img_height,img_width,_ = dataset.shape
    processed_dataset = np.zeros((n_imgs,img_height,img_width,1))
    for i in range(len(dataset)):
        img = dataset[i]
        processed_dataset[i,:,:,:] = img/255.-0.5
    return processed_dataset

X_train, X_val = preprocessed(X_train), preprocessed(X_val)
y_train, y_val = to_categorical(y_train, n_class), to_categorical(y_val, n_class)

### 【练习】CNN模型架构


创建一个卷积神经网络来对交通标志进行分类。在你代码块的最后，执行 `model.summary()` 来输出你模型的总结信息。
    
我们已经帮你导入了一些所需的 Python 库，如有需要你可以自行导入。如果你在过程中遇到了困难，请查阅[Keras英文文档](https://keras.io/)。

在这里可以尝试采用经典的 LeNet-5 架构，如下图所示：

![Sample Lenet](images/lenet.png)

参考结构如下：

01. 5x5 convolution (32x32x1 in, 28x28x6 out)
02. ReLU
03. 2x2 max pool (28x28x6 in, 14x14x6 out)
04. 5x5 convolution (14x14x6 in, 10x10x16 out)
05. ReLU
06. 2x2 max pool (10x10x16 in, 5x5x16 out)
07. 5x5 convolution (5x5x16 in, 1x1x400 out)
08. ReLu
09. Flatten layers from numbers 8 (1x1x400 -> 400) and 6 (5x5x16 -> 400)
10. Concatenate flattened layers to a single size-800 layer
11. Dropout layer
12. Fully connected layer (800 in, 43 out)

>**Note: 这里必须采用函数式写法构建 CNN [参考链接](https://keras.io/models/model/)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Input, Dropout, Flatten, Dense, Concatenate
from keras.models import Model

input_tensor = Input(X_train.shape[1:])
x = input_tensor

### TODO: 定义你的网络架构
                 
model.summary()

---
<a id='step4'></a>
## 步骤4：模型训练

模型训练过程中我们使用了早期停止来防止过拟合，并使用学习率衰减寻找局部最优解。

---
<a id='question2'></a>  

### __问题 2:__ 

简要概括早期停止与学习率衰减的原理。

__回答:__ 

In [ ]:
from keras_tqdm import TQDMNotebookCallback
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model.compile(Adam(lr=2e-3), loss='categorical_crossentropy', metrics=['accuracy'])

model_name = './best_model.h5'
callbacks = [EarlyStopping(monitor='val_loss', patience=9, verbose=2), # 早期停止
             ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.25, min_lr=1e-5, verbose=2), # 学习率衰减
             ModelCheckpoint(model_name, save_best_only=True, save_weights_only=True), # 储存最佳模型
             TQDMNotebookCallback(leave_inner=True, metric_format='{value:0.3f}')]

history = model.fit(X_train, y_train, epochs = 200, batch_size = 256, verbose = 0,
                    validation_data=(X_val,y_val), callbacks = callbacks)

In [ ]:
# 绘制损失函数、精确率与学习速率的变化曲线
with mpl.rc_context(rc={'font.family': 'serif', 'font.size': 11}):
    fig = plt.figure(figsize=(20,5))
    ax1 = fig.add_subplot(131)
    ax1.set_xlabel('Epoch')
    ax1.set_title('Loss')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['train', 'val'], loc='upper right')
    ax2 = fig.add_subplot(132)
    ax2.set_xlabel('Epoch')
    ax2.set_title('Accuracy')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.legend(['train', 'val'], loc='lower right')
    ax3 = fig.add_subplot(133)
    ax3.set_xlabel('Epoch')
    ax3.set_title('Learning Rate')
    plt.plot(history.history['lr'])

---
<a id='step5'></a>
## 步骤5 测试你的算法

将对测试集的预测结果储存在 `submit.csv` 中，并提交至邮箱 hzhmoon1217@163.com。

In [ ]:
### 加载最佳模型
model.load_weights('./best_model.h5')

test_pred = model.predict(preprocessed(test['features']))
predictions = np.array([np.argmax(i) for i in test_pred])

### 写入CSV文件
with open('./submit.csv', 'w') as f:
    f.write('Image,ClassId\n')
    for i in range(len(test['features'])):
        f.write(str(i).zfill(5) + '.ppm,' + str(predictions[i]) + '\n')
    f.close()

---
<a id='question3'></a>  

### __问题 3:__ 

思考：采用哪些方法可以进一步提高模型的准确度？

__回答:__ 